<h4>Import required libraries</h4>

In [167]:
import os
import sys
import csv

import time
import warnings
import datetime
import random
import pickle
import numpy as np
import pandas as pd


import torch
from torch import nn
import torch.multiprocessing
import torch.distributed as dist
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# import torchvision
# from torchvision import transforms

from transformers import AutoModel, AutoModelWithLMHead, AutoModelForCausalLM, pipeline
#from datasets import load_dataset, load_from_disk
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from accelerate import Accelerator


<h4>Set required env and global variables</h4>

In [47]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["TORCH_USE_CUDA_DSA"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "6,7"
os.environ["RANK"] = "0"
os.environ["LOCAL_RANK"] = "0"
os.environ["WORLD_SIZE"] = "2"
os.environ["MASTER_PORT"] = "29580"
os.environ["MASTER_ADDR"] = "127.0.0.1"

In [88]:
RANDOM_SEED = 7
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

Using device: cuda


<h2>Data-preprocessing<h/2>

Visualize individual databases and then combine them

In [24]:
true_df = pd.read_csv("True.csv")
fake_df = pd.read_csv("Fake.csv")
mix_df = pd.read_csv("WELFake_Dataset.csv")

In [25]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
mix_df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [7]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [8]:
print(len(true_df), len(fake_df), len(mix_df))

21417 23481 72134


Looking at the layout of the datasets, the common features are 'text' and 'title'. Combining these 3 different datasets into a test and train dataset.

// TODO : Can 'title' can be included as a feature?

WEL_database has their label flipped. So need ot reverse that in the following helper funtion. 

// TODO : explain why they're flipped

In [159]:
def get_vanilla_datasets():
    true_df = pd.read_csv("True.csv")
    fake_df = pd.read_csv("Fake.csv")
    train_df = pd.read_csv("WELFake_Dataset.csv")

    # drop columns that will not be used.
    true_df = true_df.drop(columns=["subject", "date", "title"])
    fake_df = fake_df.drop(columns=["subject", "date", "title"])
    train_df = train_df.drop(columns=["Unnamed: 0", "title"])    

    # add a 'label' columns
    true_df['label'] = 1
    fake_df['label'] = 0

    # concatinate 'true' and 'fake' news datasets
    combined_df = pd.concat([true_df, fake_df])

    # clean dataframes 
    combined_df = combined_df.dropna()
    train_df = train_df.dropna()

    # flip labels for mix_df
    train_df['label'] = train_df["label"] ^ 1 # XOR operation flips 1 and 0

    # shuffle databases
    train_df = train_df.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)
    combined_df = combined_df.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)

    train_df.name = "vanilla training dataset"
    combined_df.name = "vanilla testing dataset"

    return train_df, combined_df

In [160]:
train_df, test_df = get_vanilla_datasets()

In [86]:
train_df.head()

,text,label
0,No comment is expected from Barack Obama Membe...,0
1,Did they post their votes for Hillary already?,0
2,"Now, most of the demonstrators gathered last ...",0
3,A dozen politically active pastors came here f...,1
4,"The RS-28 Sarmat missile, dubbed Satan 2, will...",0


In [126]:
test_df.head()

,text,label
0,The parent company for the conservative Fox Ne...,0
1,BERLIN (Reuters) - U.S. President Donald Trump...,1
2,NEW DELHI (Reuters) - India could take up the ...,1
3,A member of the House Intelligence Committee i...,0
4,When your presidential candidate is supported ...,0


In [37]:
true_sample = true_df['text'][0]
fake_sample = fake_df['text'][0]

LLM-Stylize articles to generate adversarial input. 

In [3]:
def build_real_prompts(passages):
    messages = []
    for passage in passages:

        sys_message = "You are a writing AI assistant. Rewrite the provided text with official language. Respond only with the rewritten passage."
        user_message = "Respond only with the rewritten passage. Here is the passage: \n {}. ".format(passage)

        message = [
            { "role" : "system", "content" : sys_message},
            { "role" : "user", "content" : user_message}
            ]
        messages.append(message)
    return messages

def build_fake_prompts(passages):
    messages = []
    for passage in passages:

        sys_message = "You are a writing AI assistant. Rewrite the provided text like a scammer. Respond only with the rewritten passage."
        user_message = "Respond only with the rewritten passage. Here is the passage: \n {}. ".format(passage)

        message = [
            { "role" : "system", "content" : sys_message},
            { "role" : "user", "content" : user_message}
            ]
        messages.append(message)
    return messages

In [25]:
model_id = "meta-llama/Llama-3.2-3B-Instruct" # better adherence to instructions

pipe = pipeline(
    task="text-generation", 
    model=model_id, 
    model_kwargs={"torch_dtype" : torch.bfloat16}, 
    device_map="auto",
    max_new_tokens=512,
    return_full_text=False
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [65]:
# Helper functions to generate LLm stylized datasets.

def get_batch_of_articles(index, data):
    batch_size = 10
    ret = []
    start = index*batch_size
    for i in range(batch_size):
        ret.append(data['text'][start+i])
    return ret

def get_responses(messages):
    llm_responses = []
    for message in messages:
        llm_response = message[0]['generated_text']
        llm_responses.append(llm_response)
    return llm_responses

def generate_llm_stylized(makeFake):
    filename = "fake2real_output" if makeFake else "rea2fake_output"
    header = ['text']
    for i in range(20):
        articles = get_batch_of_articles(i, fake_df)
        msgs = []
        if makeFake:
            msgs = build_real_prompts(articles)
        else:
            msgs = build_fake_prompts(articles)
        responses = pipe(msgs)
        llm_articles = get_responses(responses)
        f_name = filename+"_"+str(i)+".csv"
        with open(f_name, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(header)
            for item in llm_articles:
                writer.writerow([item])


In [10]:
# generate_llm_stylized(makeFake=True)

In [67]:
def generate_llm_stylized_v2(makeFake):
    filename = "fake2real_output" if makeFake else "real2fake_output"
    db = fake_df if makeFake else true_df
    header = ['text']
    for i in range(20):
        articles = get_batch_of_articles(i, db)
        msgs = []
        if makeFake:
            msgs = build_real_prompts(articles)
        else:
            msgs = build_fake_prompts(articles)
        responses = pipe(msgs)
        llm_articles = get_responses(responses)
        f_name = filename+"_"+str(i)+".csv"
        with open(f_name, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(header)
            for item in llm_articles:
                writer.writerow([item])

In [11]:
# generate_llm_stylized_v2(makeFake=False)

In [38]:
real_texts = []
fake_texts = []
for i in range(200):
    real_texts.append(true_df['text'][i][0:500])
    fake_texts.append(fake_df['text'][i][0:500])

In [39]:
f2r_csv_files = [os.path.join(".", f) for f in os.listdir(".") if f.startswith("fake2real")]
r2f_csv_files = [os.path.join(".", f) for f in os.listdir(".") if f.startswith("real2fake")]

In [41]:
f2r_combined_df = pd.concat([pd.read_csv(file) for file in f2r_csv_files])
r2f_combined_df = pd.concat([pd.read_csv(file) for file in r2f_csv_files])

f2r_combined_df['label'] = 0
r2f_combined_df['label'] = 1

In [162]:
def get_adversarial_dataset():

    r2f_combined_df.to_csv("real2fake.csv")
    f2r_combined_df.to_csv("fake2real.csv")

    real2fake_df = pd.read_csv("real2fake.csv", index_col=0).sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)
    fake2real_df = pd.read_csv("fake2real.csv", index_col=0).sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)
    llm_df = pd.concat([real2fake_df, fake2real_df]).sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)

    llm_df.name = "LLM Stylized dataset"
    real2fake_df.name = "Real2Fake LLM dataset"
    fake2real_df.name = "Fake2Real LLM dataset"

    return llm_df, real2fake_df, fake2real_df

In [163]:
llm_df, r2f_df, f2r_df = get_adversarial_dataset()

<h2>Test a variety of models to evaluate their performance on datasets</h2>

In [155]:
from transformers import pipeline
MODEL_1 = "jy46604790/Fake-News-Bert-Detect"
MODEL_2 = "winterForestStump/Roberta-fake-news-detector"

In [164]:
def model_input(model_id, text):
    # if model_id == MODEL_1:
    #     text = text[0:500]
    return text

def model_prediction(clf_model, text):
    # if clf_model.model.name_or_path == MODEL_1:
        # print(type(text))
    encoding = clf_model.tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to(DEVICE)
    output = clf_model.model(**encoding)
    logits = output.logits
    probs = F.softmax(logits, dim=-1)
    predicted_label = torch.argmax(probs, dim=-1)
    predicted_label = predicted_label.item()

    return predicted_label


def establish_performance(model_id, df, debug=False):
    clf = pipeline("text-classification", model=model_id, tokenizer=model_id, device=DEVICE)
    count = 0
    max_len = len(df)
    if debug:
        max_len = 500
    for i in range(max_len):
        text = model_input(model_id=model_id, text=df['text'][i])
        res = model_prediction(clf_model=clf, text=text)
        # print(res, _df['label'][i])
        if res == df['label'][i]:
            count += 1
    acc = (count * 100) / max_len
    print("Accuracy of model {} on {} is {}".format(model_id, df.name, acc))
    return acc

In [165]:
establish_performance(MODEL_1, train_df, debug=True)
establish_performance(MODEL_1, test_df, debug=True)
establish_performance(MODEL_1, r2f_df)
establish_performance(MODEL_1, f2r_df)
establish_performance(MODEL_1, llm_df)

Accuracy of model jy46604790/Fake-News-Bert-Detect on vanilla trainset is 85.2
Accuracy of model jy46604790/Fake-News-Bert-Detect on vanilla test set is 100.0
Accuracy of model jy46604790/Fake-News-Bert-Detect on Real2Fake LLM dataset is 2.5
Accuracy of model jy46604790/Fake-News-Bert-Detect on Fake2Real LLM dataset is 99.5
Accuracy of model jy46604790/Fake-News-Bert-Detect on LLM Stylized dataset is 51.0


51.0

In [166]:
establish_performance(MODEL_2, train_df, debug=True)
establish_performance(MODEL_2, test_df, debug=True)
establish_performance(MODEL_2, r2f_df)
establish_performance(MODEL_2, f2r_df)
establish_performance(MODEL_2, llm_df)

Accuracy of model winterForestStump/Roberta-fake-news-detector on vanilla trainset is 70.8
Accuracy of model winterForestStump/Roberta-fake-news-detector on vanilla test set is 84.0
Accuracy of model winterForestStump/Roberta-fake-news-detector on Real2Fake LLM dataset is 10.0
Accuracy of model winterForestStump/Roberta-fake-news-detector on Fake2Real LLM dataset is 57.5
Accuracy of model winterForestStump/Roberta-fake-news-detector on LLM Stylized dataset is 33.75


33.75

<h2> Finetune BERT </h2>

In [169]:
import pandas as pd
from transformers import BertTokenizer
from datasets import Dataset

# Step 1: Convert Pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Step 2: Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Step 3: Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

# Step 4: Apply the tokenization to the datasets using `map()` with `batched=True`
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Step 5: Rename the label column to 'labels' (required by the model)
train_dataset = train_dataset.rename_column("label", "labels")
test_dataset = test_dataset.rename_column("label", "labels")

# Step 6: Set the format to PyTorch (torch tensors)
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Now your datasets are ready for use with Hugging Face's Trainer


Map:   0%|          | 0/72095 [00:00<?, ? examples/s]

Map:   0%|          | 0/44898 [00:00<?, ? examples/s]

In [170]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [171]:
training_args = TrainingArguments(
    output_dir='./results',          # Output directory for model checkpoints
    evaluation_strategy="epoch",     # Evaluate once per epoch
    learning_rate=2e-5,              # Learning rate for optimization
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    num_train_epochs=3,              # Number of epochs
    weight_decay=0.01,               # Weight decay for regularization
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,                # Log every 10 steps
)

/home/grads/a/atkulkarni/MIPS/mips/lib/python3.12/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


DistStoreError: Timed out after 1801 seconds waiting for clients. 1/2 clients joined.

In [ ]:
trainer = Trainer(
    model=model,                         # The pre-trained model
    args=training_args,                  # Training arguments
    train_dataset=train_dataset,         # Training dataset
    eval_dataset=test_dataset,           # Evaluation dataset
    tokenizer=tokenizer,                 # Tokenizer for padding/truncation
)

In [ ]:
trainer.train()

In [ ]:
results = trainer.evaluate()
print(results)